Write a simple Hamming encoder program in Python, which, when given a 4-bit binary value, returns the resulting 7-bit binary vector codeword. 
Also implement the parity check functionality to see if there are any errors, that is to check whether H ∗ cw = 0 holds, where is zero vector.

Hamming Class or encoding, decoding, parity check

In [83]:
import numpy as np
import random

In [166]:
class Support:
    """
    Support class with utility methods for Hamming Code operations.
    """

    @staticmethod
    def create_random_vector():
        """
        Generate a random 4-bit vector.

        Returns:
            np.ndarray: Random 4-bit vector.
        """
        # Use NumPy's randint to generate a random vector of size 4 with elements in the range [0, 2).
        # This range ensures the generated bits are either 0 or 1.
        random_vector = np.random.randint(0, 2, size=4)
        return random_vector

    @staticmethod
    def check_input(codeword, input_length):
        """
        Check if the input satisfies the requirements for Hamming Code operations.

        Args:
            codeword (int, list, str, np.ndarray): Input to be validated.
            input_length (int): Expected length of the input array.

        Returns:
            np.ndarray: Validated and converted input array.

        Raises:
            TypeError: If the input type is not int, list, str, or np.ndarray.
            ValueError: If input requirements are not met.
        """
        # Ensure that the input 'codeword' is of a valid type: integer, list, or NumPy array
        if not isinstance(codeword, (list, np.ndarray, str)):
            # Raise a TypeError if the input is not one of the allowed types: integer, list, string, or NumPy array
            raise TypeError("The input must be a list, a string, or a NumPy array.")

        # If 'codeword' is of string type, remove any white spaces
        if isinstance(codeword, str):
            codeword = codeword.replace(" ", "")
            # Verify that the string comprises only numerical digits
            if not codeword.isdigit():
                raise ValueError("String input must contain only digits.")
            # Convert the string of digits into a list of integers
            codeword = [int(digit) for digit in codeword]

        # Ensure 'codeword' is represented as a NumPy array and attempting conversion if needed
        if not isinstance(codeword, np.ndarray):
            try:
                codeword = np.array(codeword)
            except Exception as e:
                raise ValueError(f"Failed to convert input to a NumPy array: {e}")

        # Ensure that the size of 'codeword' matches the expected input length
        if codeword.size != input_length:
            raise ValueError(f"The input array must have a size of {input_length}.")

        # Ensure that the elements of 'codeword' are integers
        if not np.issubdtype(codeword.dtype, np.integer):
            raise ValueError("The input array must contain only integers.")

        # Ensure that the elements of 'codeword' are either 0 or 1 (binary digits)
        if not np.all(np.isin(codeword, [0, 1])):
            raise ValueError("The input array must contain only 0s and 1s.")

        return codeword

    @staticmethod
    def bitflip_rand(codeword, number_of_bits_to_flip):
        """
        Flip a specified number of random bits in the input array 'codeword'.

        Args:
            codeword (np.ndarray): Input array.
            number_of_bits_to_flip (int): Number of bits to flip.

        Returns:
            np.ndarray: Input array with flipped bits.

        Raises:
            ValueError: If the number_of_bits_to_flip is greater than 2.
        """
        # Validate and convert the input array to ensure it adheres to the Hamming Code requirements
        codeword = Support.check_input(codeword, 7)
        
        # Validate the number_of_bits_to_flip to ensure it aligns with the limitations of the Hamming Code algorithm
        if number_of_bits_to_flip > 2:
            raise ValueError("The Hamming Code only allows detecting at most two bitflips.")
        elif number_of_bits_to_flip <= 0:
            raise ValueError("The number of bits should be 1 or 2.")
        else:
            flipped_bits = set()
        
            while True:
                # Randomly choose an index i in the range [0, 6]
                i = random.randint(0, 6)
                # Flip the bit at index i if it has not been flipped before
                if i not in flipped_bits:
                    flipped_bits.add(i)
                    # Create a copy of the array before modifying it
                    codeword = codeword.copy()
                    codeword[i] = 1 - codeword[i]
                # Break the loop if the number of flipped bits matches the number_of_bits_to_flip
                if len(flipped_bits) == number_of_bits_to_flip:
                    break
        return codeword

    @staticmethod
    def bitflip_specific(codeword, bit_to_flip):
        """
        Flip a specific bit in the input array.

        Args:
            codeword (np.ndarray): Input array.
            bit_to_flip (int): Index of the bit to flip.

        Returns:
            np.ndarray: Input array with the specified bit flipped.

        Raises:
            ValueError: If bit_to_flip is greater than 6 (out of bounds).
        """
        # Validate and convert the input array to ensure it adheres to the Hamming Code requirements
        codeword = Support.check_input(codeword, 7)
        # Ensure the specified bit index is within the valid range (0 to 6 inclusive)
        if bit_to_flip > 6:
            raise ValueError("The codeword has only 7 bits.")
        else:
            # Flip the bit at specified index in the input array
            codeword[bit_to_flip] = 1 - codeword[bit_to_flip]
        return codeword


In [15]:
ex1 = Support()
victor = ex1.create_random_vector()
print(victor)
ex1.check_input(1010101, 7)
ex1.bitflip_rand(1010101, 2)

[0 1 1 1]


array([0, 0, 0, 0, 1, 0, 1])

In [85]:
class Hamming:
    """
    Class for encoding, decoding, and checking Hamming codes.
    """
    # GeneratorMatrix (G)
    G = np.array([[1, 1, 0, 1],
                  [1, 0, 1, 1],
                  [1, 0, 0, 0],
                  [0, 1, 1, 1],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    
    # ParityCheckMatrix (H)
    H = np.array([[1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 1, 0, 0, 1, 1],
                  [0, 0, 0, 1, 1, 1, 1]])
    
    # DecoderMatrix (R)
    R = np.array([[0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 0, 0, 1]])

    @staticmethod
    def encoder(input):
        """
        Encode a 4-bit vector into a 7-bit codeword using the Generator Matrix (G).

        Args:
            input (int, list, str, np.ndarray): 4-bit vector to be encoded.

        Returns:
            np.ndarray: 7-bit codeword.
        """
        # Validate and convert the input array to ensure it adheres to the Hamming Code requirements
        input = Support().check_input(input, 4)
        # Use the Generator Matrix (G) to perform the encoding
        codeword = np.matmul(Hamming.G, input) % 2
        # Display the resulting 7-bit codeword
        print(f'The 7-bit codeword is {codeword}.')
        return codeword
    
    @staticmethod
    def parity_check(codeword):
        """
        Perform a parity check on a 7-bit codeword using the Parity-check Matrix (H).

        Args:
            codeword (int, list, str, np.ndarray): 7-bit codeword to be checked.

        Returns:
            dict: A dictionary containing the results of the parity check analysis, including the error syndrome,
            sum of the error syndrome, and the status of the error.
        """
        # Validate and convert the input array to ensure it adheres to the Hamming Code requirements
        codeword = Support.check_input(codeword, 7)
        # Calculate the error syndrome using the Parity-check Matrix (H)
        error_syndrome = np.matmul(Hamming.H, codeword) % 2
        print(f'The error vector is {error_syndrome}.')
        # Calculate the sum of the error syndrome elements
        sum_error_syndrome = np.sum(error_syndrome)

        # Create a dictionary to store the results of the parity check analysis
        results_dict = {'Error syndrome': error_syndrome.tolist(), 'Sum of the Error syndrome': sum_error_syndrome}

        # Analyze the error syndrome and provide appropriate messages
        if sum_error_syndrome == 0:
            results_dict['Status'] = 'No Error'
            print('There was no error occurring upon code transmission.')
        elif sum_error_syndrome == 1:
            results_dict['Status'] = 'Uncorrectable Error. The bitflip cannot be recovered'
            print('There was an uncorrectable error in the codeword due to an error in a parity bit or two errors in data bits.')
        # If the sum of error syndrome elements is 2 or 3, the error can be corrected
        else: 
            # Transpose the matrix to facilitate comparison of error syndrome with transposed rows
            transposed_parity = Hamming.H.transpose()
            # Convert the matrix to a list for index function usage
            list_parity = transposed_parity.tolist()
            # Find the index of the error syndrome in the transposed matrix
            list_parity_index = list_parity.index(error_syndrome.tolist())
            # Add 1 to the index to get the correct row number, indicating the position of bitflip.
            bitflip_pos = list_parity_index + 1
            results_dict['Status'] = 'Correctable Error. The bitflip can be recovered'
            print(f'There was an error occurring upon code transmission and the error can be corrected.\nThe Bitflip occurred at the {bitflip_pos}th position.')
        return results_dict

    @staticmethod
    def decoder(codeword):
        """
        Decode a 7-bit codeword into the original 4-bit vector, correcting errors if possible.

        Args:
            codeword (int, list, str, np.ndarray): 7-bit codeword to be decoded.

        Returns:
            np.ndarray: The original 4-bit vector obtained after decoding the input codeword.
        """
        # Validate and convert the input array to ensure it adheres to the Hamming Code requirements
        codeword = Support().check_input(codeword, 7)
        # Calculate the error syndrome by multiplying the Parity Check Matrix (H) with the codeword
        error_syndrome = np.matmul(Hamming.H, codeword) % 2
        sum_error_syndrome = np.sum(error_syndrome)

        binary_decoded = None

        if sum_error_syndrome == 0:
            print('There is no bitflip error in the codeword to be corrected.')
            # Use the Decoder Matrix (R) to obtain the original 4-bit vector
            binary_decoded = np.matmul(Hamming.R, codeword) % 2
            print(f'The original 4-bit vector is {binary_decoded}.')
        elif sum_error_syndrome == 1:
            print('There was an error in the codeword due to an error in a parity bit or two errors in data bits. In both cases, the codeword cannot be decoded.')
        # If the sum_error_syndrome is either 2 or 3, indicating the codeword can be decoded
        else: 
            # Transpose the matrix to be able to compare our error syndrome with the rows of the transposed matrix
            transposed_parity = Hamming.H.transpose()
            # Convert the matrix to a list for index function usage
            list_parity = transposed_parity.tolist()
            # Find the index of the error syndrome in the transposed matrix
            list_parity_index = list_parity.index(error_syndrome.tolist())
            # Add 1 to the index to get the correct row number, indicating the position of bitflip.
            bitflip_pos = list_parity_index + 1
            print(f'The bit at position {bitflip_pos} was flipped.')

            # Correct the bit at the identified position of the bitflip
            if codeword[bitflip_pos - 1] == 0:
                codeword[bitflip_pos - 1] = 1
            else:
                codeword[bitflip_pos - 1] = 0
            print(f'The corrected codeword is {codeword}.')
            # Use the Decoder Matrix (R) to obtain the original 4-bit vector
            binary_decoded = np.matmul(Hamming.R, codeword) % 2
            print(f'The original 4-bit vector is {binary_decoded}.')
        return binary_decoded

In [162]:
# Test 1 (Input given by user, encode, no bit flipped, parity check, decode)
test1_sequence = [int(i) for i in str(1001)]
test1 = Hamming()
codeword1 = test1.encoder(test1_sequence)
test1.parity_check(codeword1)
test1.decoder(codeword1)

The 7-bit codeword is [0 0 1 1 0 0 1].
The error vector is [0 0 0].
There was no error occurring upon code transmission.
There is no bitflip error in the codeword to be corrected.
The original 4-bit vector is [1 0 0 1].


array([1, 0, 0, 1])

In [165]:
# Test 2 (Input given by user, encode, flip 3rd bit (Index 2), parity check, decode)
test2_sequence = [int(i) for i in str(1001)]
test2 = Hamming()
codeword2 = test2.encoder(test2_sequence)
codeword2 = Support.bitflip_specific(codeword2, 2)
test2.parity_check(codeword2)
test2.decoder(codeword2)

The 7-bit codeword is [0 0 1 1 0 0 1].
The error vector is [1 1 0].
There was an error occurring upon code transmission and the error can be corrected.
The Bitflip occurred at the 3th position.
The bit at position 3 was flipped.
The corrected codeword is [0 0 1 1 0 0 1].
The original 4-bit vector is [1 0 0 1].


array([1, 0, 0, 1])

In [167]:
# Test 3 (Input given by user, encode, flip two bits at random, parity check, (decode, not possible))
# Will print messages (for parity check and decode functions) that the error is not correctable and hence,
# cannot be decoded
test3_sequence = [1, 0, 0, 1]
test3 = Hamming()
codeword3 = test3.encoder(test3_sequence)
codeword3 = Support.bitflip_rand(codeword3, 2)
print(codeword3)
test3.parity_check(codeword3)
test3.decoder(codeword3)

The 7-bit codeword is [0 0 1 1 0 0 1].
[1 0 0 1 0 0 1]
The error vector is [0 1 0].
There was an uncorrectable error in the codeword due to an error in a parity bit or two errors in data bits.
There was an error in the codeword due to an error in a parity bit or two errors in data bits. In both cases, the codeword cannot be decoded.


In [ ]:
#Test 4 (Input given by user, encode, flip three bits at random (not possible), parity check, (decode, not possible))
# Will raise ValueError since the Hamming code only allows detection of at most two bitflips
test4_sequence = [int(i) for i in str(1001)]
test4 = Hamming()
codeword4 = test4.encoder(test4_sequence)
codeword4 = Support.bitflip_rand(codeword4, 3)
print(codeword4)


In [149]:
# Test 5 (Input given by user, encode, no bit flipped, parity check, decode)
# Will work as intended
test5_sequence = [1, 1, 0, 1]
test5 = Hamming()
codeword5 = test5.encoder(test5_sequence)
test5.parity_check(codeword5)
codeword5 = test5.decoder(codeword5)

The 7-bit codeword is [1 0 1 0 1 0 1].
The error vector is [0 0 0].
There was no error occurring upon code transmission.
There is no bitflip error in the codeword to be corrected.
The original 4-bit vector is [1 1 0 1].


In [ ]:
# Test 6 (Input given by user is too long (5 digits), encode, no bit flipped, parity check, decode)
# Will raise ValueError since (7,4)-algorithm merely allows for encoding 4-bit vectors (not 5-bit vectors)
test6_sequence = [1, 0, 0, 1, 1]
test6 = Hamming()
codeword6 = test6.encoder(test6_sequence)

In [144]:
# Test 7 (Input is randomly created, encode, no bit flipped, parity check, decode)
# Will work as intended
# Other functionalities that could be tested, regarding bitflip creation, etc., would work accordig to other test above and hence, as intended
test7_sequence = Support.create_random_vector()
print(test7_sequence)
test7 = Hamming()
codeword7 = test7.encoder(test7_sequence)
test7.parity_check(codeword7)
test7.decoder(codeword7)

[0 1 0 1]
The 7-bit codeword is [0 1 0 0 1 0 1].
The error vector is [0 0 0].
There was no error occurring upon code transmission.
There is no bitflip error in the codeword to be corrected.
The original 4-bit vector is [0 1 0 1].


array([0, 1, 0, 1])

In [140]:
# Test 8 (Input is only 0s)
test8_sequence = [0, 0, 0, 0]
test8 = Hamming()
codeword8 = test8.encoder(test8_sequence)
test8.parity_check(codeword8)
test8.decoder(codeword8)

The 7-bit codeword is [0 0 0 0 0 0 0].
The error vector is [0 0 0].
There was no error occurring upon code transmission.
There is no bitflip error in the codeword to be corrected.
The original 4-bit vector is [0 0 0 0].


array([0, 0, 0, 0])

In [136]:
# Test 9 (Input only 0s, but directly for the parity check with 7 digits)
# Valid output, i.e., function can potentially be used without prior encoding
test9_sequence = [0, 0, 0, 0, 0, 0, 0]
test9 = Hamming()
test9.parity_check(test9_sequence)

The error vector is [0 0 0].
There was no error occurring upon code transmission.


{'Error syndrome': [0, 0, 0],
 'Sum of the Error syndrome': 0,
 'Status': 'No Error'}

In [134]:
# Test 10 (Input is only 0s as string)
test10_sequence = "0000"
test10 = Hamming()
test10.encoder(test10_sequence)

The 7-bit codeword is [0 0 0 0 0 0 0].


array([0, 0, 0, 0, 0, 0, 0])

In [133]:
# Test 11 (Input is only 1s)
test11_sequence = [1, 1, 1, 1]
test11 = Hamming()
codeword11 = test11.encoder(test11_sequence)
Support.check_input(codeword11, 7)
test11.parity_check(codeword11)
test11.decoder(codeword11)

The 7-bit codeword is [1 1 1 1 1 1 1].
The error vector is [0 0 0].
There was no error occurring upon code transmission.
There is no bitflip error in the codeword to be corrected.
The original 4-bit vector is [1 1 1 1].


array([1, 1, 1, 1])

In [155]:
# Test 12 (Input is leading 0 (i.e., 0110, wihch cannot be handeld by Python)
# Will throw error message since leading zeros for integers are not permitted
# Moreover, in Support class above specified, that not of valid input type
test12_sequence = 0110
test12 = Hamming()
test12.encoder(test12_sequence)
print(test12)

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (976925702.py, line 4)

In [ ]:
# Test 13 (Input given as an integer, encode)
# Will throw an error since input not given as valid format (list, string, np.array)
test13_sequence = 1001
test13 = Hamming()
codeword13 = test13.encoder(test13_sequence)

In [168]:
# Cases for testing the check_input function
print(Support.check_input([1, 0, 1, 1], 4))
print(Support.check_input([1, 0, 0, 1, 1], 5)) # array can be created (for other use cases the user wants to test), but not encoded due to input check at the beginning of encoder function
print(Support.check_input(np.array([1, 0, 1, 1]), 4))
print(Support.check_input("1 0 1 1", 4))

[1 0 1 1]
[1 0 0 1 1]
[1 0 1 1]
[1 0 1 1]


In [169]:
# Tests for testing functions of Support class indivdually
sup1 = Support()
first_vector = sup1.create_random_vector()
print(first_vector)
sup1.check_input(Hamming.encoder(first_vector), 7)
sup1.bitflip_rand(Hamming.encoder(first_vector), 2)

[0 0 0 1]
The 7-bit codeword is [1 1 0 1 0 0 1].
The 7-bit codeword is [1 1 0 1 0 0 1].


array([1, 0, 0, 1, 1, 0, 1])